In [28]:
import pandas as pd
import pymysql
from tqdm import tqdm

## Find the Synonym Of The Medication Class

In [7]:
Medication = pd.read_csv("RA medication class.csv")

In [8]:
Medication = Medication.drop_duplicates(subset=['MedicationCui', 'ClassCui'])

In [9]:
database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
cursor = database.cursor()

In [10]:
command = '''select distinct cui2,str
from mrrel,mrconso
where cui1 = %s and cui2 != %s and cui = cui2 and (rel = "SY" or rela = "mapped_to");'''

In [11]:
Data = pd.DataFrame(columns=["MedicationCui","MedicationStr","ClassCui","ClassStr","ClassCui(Synonym)","ClassStr(Synonym)"])

for index,row in tqdm(Medication.iterrows()):
    cursor.execute(command,(row["ClassCui"],row["ClassCui"]))
    result = cursor.fetchall()
    if(result == ()):
        Data = pd.concat([Data,row.to_frame().T],ignore_index=True)
        continue
    Temp = pd.DataFrame(result,columns=["ClassCui(Synonym)","ClassStr(Synonym)"])
    duplicated_row = pd.concat([row.to_frame().T] * len(Temp), ignore_index=True)
    combined = pd.concat([duplicated_row.reset_index(drop=True), Temp.reset_index(drop=True)], axis=1)
    Data = pd.concat([Data,combined],ignore_index=True)

577it [6:46:21, 42.26s/it] 


In [12]:
cursor.close()
database.close()

In [13]:
Data.drop_duplicates(subset=['ClassCui(Synonym)'])

,MedicationCui,MedicationStr,ClassCui,ClassStr,ClassCui(Synonym),ClassStr(Synonym)
0,C0004057,aspirin,C0002771,Analgesics,NaN,NaN
1,C0004057,aspirin,C0003211,"Anti-Inflammatory Agents, Non-Steroidal",C3536840,Nonsteroidal Anti-inflammatory Drug [EPC]
12,C0004057,aspirin,C0004057,aspirin,C0050536,acetylsalicylic anhydride
14,C0004057,aspirin,C0004057,aspirin,C0050537,acetylsalicylsalicylic acid
15,C0004057,aspirin,C0004057,aspirin,C0051254,aloxiprin
...,...,...,...,...,...,...
65202,C4726929,upadacitinib,C0019398,Heterocyclic Compounds,C4309267,"3,4-9,10-13,14-tribenzo-1,12-diaza-5,8-dioxacy..."
65203,C4726929,upadacitinib,C0019398,Heterocyclic Compounds,C4309753,tetraaza(1.1.1.1)cyclophane
65204,C4726929,upadacitinib,C0019398,Heterocyclic Compounds,C5420928,deucravacitinib
65205,C4726929,upadacitinib,C0019398,Heterocyclic Compounds,C5565551,SBI-797812


In [14]:
Medication = Data.drop_duplicates(subset=['MedicationCui','ClassCui','ClassCui(Synonym)'])

In [15]:
Data.drop_duplicates(subset=['MedicationCui','ClassCui','ClassCui(Synonym)']).to_csv("SynonymClass(RA).csv",index=False,encoding='utf-8-sig')

In [16]:
Medication

,MedicationCui,MedicationStr,ClassCui,ClassStr,ClassCui(Synonym),ClassStr(Synonym)
0,C0004057,aspirin,C0002771,Analgesics,NaN,NaN
1,C0004057,aspirin,C0003211,"Anti-Inflammatory Agents, Non-Steroidal",C3536840,Nonsteroidal Anti-inflammatory Drug [EPC]
11,C0004057,aspirin,C0003419,Antipyretics,NaN,NaN
12,C0004057,aspirin,C0004057,aspirin,C0050536,acetylsalicylic anhydride
14,C0004057,aspirin,C0004057,aspirin,C0050537,acetylsalicylsalicylic acid
...,...,...,...,...,...,...
65326,C5432357,rituximab-arrx,C0393022,rituximab,C3712411,177Lu-DOTA-rituximab
65327,C5432357,rituximab-arrx,C0393022,rituximab,C3713268,89Zr-rituximab
65328,C5432357,rituximab-arrx,C0393022,rituximab,C3884241,technetium 99m carbonyl DTPA-rituximab
65330,C5432357,rituximab-arrx,C0393022,rituximab,C4741331,Truxima


## Filter Medication Class

In [18]:
MedicationClass = pd.read_csv('RAMedicationClass.txt', sep='|', skipinitialspace=True)

In [19]:
key_cui = MedicationClass['Key Cui '].str.replace(r'\[|\]', '',regex=True).str.strip().str.split(', ').explode()
superior_key_cui = MedicationClass['Superior Key Cui '].str.replace(r'\[|\]', '',regex=True).str.strip().str.split(', ').explode()
all_cui = pd.concat([key_cui, superior_key_cui]).drop_duplicates().tolist()

In [20]:
Medication = Medication.loc[(Medication['ClassCui'].str.strip().isin(all_cui) | Medication['ClassCui(Synonym)'].str.strip().isin(all_cui))]

In [21]:
Medication

,MedicationCui,MedicationStr,ClassCui,ClassStr,ClassCui(Synonym),ClassStr(Synonym)
0,C0004057,aspirin,C0002771,Analgesics,NaN,NaN
1,C0004057,aspirin,C0003211,"Anti-Inflammatory Agents, Non-Steroidal",C3536840,Nonsteroidal Anti-inflammatory Drug [EPC]
1435,C0004320,auranofin,C0003191,Antirheumatic Agents,C0357543,Rheumatic disease suppressants
1440,C0004320,auranofin,C0003191,Antirheumatic Agents,C3536926,Antirheumatic Agent [EPC]
1457,C0004320,auranofin,C0021081,Immunosuppressive Agents,C0356955,Cytotoxic immunosuppressants
...,...,...,...,...,...,...
65326,C5432357,rituximab-arrx,C0393022,rituximab,C3712411,177Lu-DOTA-rituximab
65327,C5432357,rituximab-arrx,C0393022,rituximab,C3713268,89Zr-rituximab
65328,C5432357,rituximab-arrx,C0393022,rituximab,C3884241,technetium 99m carbonyl DTPA-rituximab
65330,C5432357,rituximab-arrx,C0393022,rituximab,C4741331,Truxima


## Filter Rxnorm Medication

In [22]:
database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
cursor = database.cursor()

In [23]:
command = '''select * from rxnconso where str = %s'''

In [24]:
for index,row in tqdm(Medication.iterrows()):
    cursor.execute(command,row["MedicationStr"])
    result = cursor.fetchall()
    if len(result) == 0:
        Medication.drop(index, inplace=True)

2692it [05:19,  8.45it/s]/var/folders/nt/61qr68k1095348fy48d1yhf80000gp/T/ipykernel_8379/852631336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Medication.drop(index, inplace=True)
2693it [05:19,  8.27it/s]/var/folders/nt/61qr68k1095348fy48d1yhf80000gp/T/ipykernel_8379/852631336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Medication.drop(index, inplace=True)
2694it [05:19,  8.26it/s]/var/folders/nt/61qr68k1095348fy48d1yhf80000gp/T/ipykernel_8379/852631336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [25]:
cursor.close()
database.close()

In [26]:
Medication.sort_index().to_csv('RA medication class(filtered).csv',encoding="utf_8_sig",index=False)

In [27]:
Medication["MedicationCui"].unique().shape

(68,)